In [1]:
# basics
import numpy as np
import pandas as pd
import datetime
import re

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]

# own data wrappers
from imp import reload
import measureclass as mc; reload(mc);
import coronadataclass as cdc; reload(cdc);

np.seterr(divide = 'ignore');


In [2]:
# load data into datawrapper classes
measure_data = mc.COVID19_measures(download_data = False, measure_level = 2, only_first_dates = True, expand_measure_names = True)
jhu_data     = cdc.CoronaData(download_data = False)

# remove and rename countries to match the JFU database and the measures database
measure_data.RemoveCountry('Diamond Princess')
measure_data.RenameCountry('France (metropole)', 'France')
measure_data.RenameCountry('South Korea', 'Korea, South')
measure_data.RenameCountry('Czech Republic', 'Czechia')


In [3]:
# helper functions

def date2vector(implementdate, start = '22/1/20', end = None, shiftdays = 0):
    # generate vector of 0s and 1s when measure is implemented or not
    starttime     = datetime.datetime.strptime(start,         '%d/%m/%y')
    if end is None:
        endtime   = datetime.datetime.today()
    else:
        endtime   = datetime.datetime.strptime(end,           '%d/%m/%y')
    implementtime = datetime.datetime.strptime(implementdate, '%d/%m/%Y')
    
    totaldays   = (endtime       - starttime).days
    measuredays = (implementtime - starttime).days
    
    vec         = np.zeros(totaldays)
    vec[min(measuredays+shiftdays,len(vec)-1):] = 1
    
    return vec


def ConvertDateFormat(date):
    m,d,y = date.split('/')
    return '{:02d}/{:02d}/{:02d}'.format(int(d),int(m),int(y))


def CleanUpMeasureName(measurename):
    # regression model formula can't contain special characters
    return ''.join([mn.capitalize() for mn in measurename.replace(',','').replace('-','').replace('/','').split(' ')])

In [4]:
def GetMeasureIDs(countrylist = None, measure_level = 2, mincount = None, extend_measure_names = False):
    if countrylist is None:
        countrylist = measure_data.countrylist # use ALL countries
    
    measurelist = {}
    
    # get all restrictions from countries
    for country in countrylist:
        country_measures = measure_data.CountryData(country, measure_level = 2, extend_measure_names = extend_measure_names)
        for measurename, initialdata in country_measures.items():
            if not measurename in measurelist.keys():
                measurelist[measurename] = 0
            measurelist[measurename] += 1
    
    if not mincount is None:
        # rebuild dict with restrictions
        measurelist = {k:v for k,v in measurelist.items() if v >= mincount}

    return measurelist


def SmoothTrajectories3(traj):
    if len(traj) > 3:
        newtraj       = np.zeros(len(traj))
        newtraj[0]    = (             2 * traj[0]    + traj[1] )/3.
        newtraj[1:-1] = (traj[0:-2] + 2 * traj[1:-1] + traj[2:])/4.
        newtraj[-1]   = (traj[-2]   + 2 * traj[-1]             )/3.
        return newtraj
    else:
        return traj


def GetCountryTrajectories(countrylist = None, data = 'Confirmed', startcases = None, maxlen = None, smooth = False):
    if countrylist is None:
        countrylist = jhu_data.countrylist
    
    trajectories = {}
    for country in [c for c in countrylist if c in jhu_data.countrylist]:
        ctraj = np.array(jhu_data.CountryData(country)[data], dtype = np.float)
        starttraj = 0
        if not startcases is None:
            starttraj = np.argmax(ctraj >= startcases)
            ctraj = ctraj[starttraj:]
            if not maxlen is None:
                ctraj = ctraj[:min(maxlen,len(ctraj))]
        trajectories[country] = {}
        if smooth:
            ctraj = SmoothTrajectories3(ctraj)
        trajectories[country]['traj'] = ctraj
        trajectories[country]['startdate'] = ConvertDateFormat(jhu_data.CountryData(country)['Date'][starttraj])
    
    return trajectories

        
def GetRegressionDF(countrylist = None, measure_level = 2, shiftdays = 0, verbose = False, maxlen = None, smooth = None):
    # construct pd.DataFrame used for regression
    
    # get trajectories and measure list for all countries in 'countrylist'
    trajectories         = GetCountryTrajectories(countrylist = countrylist, data = 'Confirmed', startcases = 30, maxlen = maxlen, smooth = smooth)
    measureIDs           = measure_data.MeasureList(countrylist = countrylist, measure_level = 2, mincount = 5)
    cleaned_measurelist  = {CleanUpMeasureName(mn):count for mn,count in measureIDs.items()}
    regressionDF         = None
    
    if verbose:
        print(measureIDs)
    
    for country in trajectories.keys():
        if country in measure_data.countrylist:

            # ********************************************
            # change observable to regress here:
            observable                  = np.diff(np.log(trajectories[country]['traj']))
            obslen                      = len(observable)
            # ********************************************
            
            DF_country = measure_data.ImplementationTable(country           = country,
                                                        measure_level     = 2,
                                                        startdate         = trajectories[country]['startdate'],
                                                        shiftdays         = shiftdays,
                                                        maxlen            = obslen,
                                                        clean_measurename = True)
            
            for measurename in DF_country.columns:
                if measurename not in measureIDs.keys():
                    DF_country.drop(labels = measurename, axis = 'columns')
            
            DF_country['Country']    = country
            DF_country['Observable'] = observable

            
            if not (np.isnan(DF_country['Observable']).any() or np.isinf(DF_country['Observable']).any()):

                if regressionDF is None:
                    regressionDF = DF_country
                else:
                    regressionDF = pd.concat([regressionDF,DF_country], ignore_index = True, sort = False)
    
    # not implemented measures should be NaN values, set them to 0
    regressionDF.fillna(0, inplace = True)
    
    return regressionDF, cleaned_measurelist



def GetCountryMasks(regrDF):
    countrylist = list(regrDF['Country'].unique())
    maskdict = {}
    for country in countrylist:
        mask = list(regrDF['Country'] == country)
        maskdict[country] = mask
    return maskdict


def CrossValidationFit(trainmodel, testmodel, header = None, alpha = 1e-5):
    results = trainmodel.fit_regularized(alpha = alpha, L1_wt = 1)
    
    result_dict = {}
    if header is not None:
        result_dict.update(header)

    test_params = []
    for paramname in testmodel.exog_names:
        if paramname in results.params.keys():
            test_params.append(results.params[paramname])
        else:
            test_params.append(0)

    result_dict['Loglike Training'] = trainmodel.loglike(results.params)
    result_dict['Loglike Test']     = testmodel.loglike(np.array(test_params))

    obs_train  = np.array(trainmodel.endog)
    obs_test   = np.array(testmodel.endog)

    pred_train = trainmodel.predict(results.params)
    pred_test  = testmodel.predict(test_params)

    result_dict['R2 Training']      = 1 - np.sum((obs_train - pred_train)**2)/np.sum((obs_train - np.mean(obs_train))**2)
    result_dict['R2 Test']          = 1 - np.sum((obs_test - pred_test)**2)/np.sum((obs_test - np.mean(obs_test))**2)

    result_dict['RSS Training']     = np.sum((obs_train - pred_train)**2)
    result_dict['RSS Test']         = np.sum((obs_test - pred_test)**2)

    result_dict.update({k:v for k,v in results.params.items()})

    return result_dict

In [ ]:
# set up of all models
# for cross-validation by excluding each country

maxshift           = 10
regrDF             = []
model_excludeC     = []
model_singleC      = []
allresults_country = None

for shiftdays in range(0, maxshift + 1):
    print('shiftdays = {}'.format(shiftdays))
    tmp, measurelist = GetRegressionDF(countrylist = measure_data.countrylist, shiftdays = shiftdays, smooth = True)
    regrDF.append(tmp)

    countrylist = tmp['Country'].unique()
    formula     = 'Observable ~ C(Country) + ' + ' + '.join(measurelist)
    
    model_excludeC.append(dict())
    model_singleC.append(dict())
    
    for country in countrylist:
        DF_excludeC = regrDF[shiftdays][regrDF[shiftdays]['Country'] != country]
        DF_singleC  = regrDF[shiftdays][regrDF[shiftdays]['Country'] == country]
        model_excludeC[shiftdays][country] = smf.ols(formula = formula, data = DF_excludeC)
        model_singleC [shiftdays][country] = smf.ols(formula = formula, data = DF_singleC)
    


In [ ]:
allresults_country = None

In [ ]:
#llres = None
for shiftdays in np.arange(maxshift + 1):
    
    print('shiftdays = {}'.format(shiftdays))
    
    for alpha in np.power(10,np.linspace(-5, -3, num = 11)):
        print('   alpha = {:.6f}'.format(alpha))
        for country in countrylist:
            result_dict = CrossValidationFit( trainmodel = model_excludeC[shiftdays][country],
                                              testmodel  = model_singleC[shiftdays][country],
                                              header     = {'shiftdays':shiftdays, 'alpha':alpha, 'excludeC':country},
                                              alpha      = alpha)
            
            if allresults_country is None:
                allresults_country = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
            else:
                allresults_country = allresults_country.append(result_dict, ignore_index = True)
        

In [6]:
# cross-validation with random sampling

maxshift     = 10
regrDF       = {}
model_train  = {}
model_test   = {}

regrDF_train = {}
regrDF_test  = {}

allresults_rndXv_chunksize = None



for shiftdays in range(5, 10):
    print('generating DF and model for shiftdays = {}'.format(shiftdays))
    regrDF[shiftdays], measurelist = GetRegressionDF( countrylist = measure_data.countrylist,
                                                      shiftdays   = shiftdays,
                                                      smooth      = True)

    countrylist = regrDF[shiftdays]['Country'].unique()
    formula     = 'Observable ~ C(Country) + ' + ' + '.join(measurelist)
    
    # generate random list of samples for 10x cross validation
    xval_count = 10
    datalen    = len(regrDF[shiftdays])
    chunklen   = np.ones(xval_count,dtype = np.int) * (datalen // xval_count)
    chunklen[:datalen%xval_count] += 1
    samples    = np.random.permutation(np.concatenate([i*np.ones(chunklen[i],dtype = np.int) for i in range(xval_count)]))

    assert len(samples) == datalen
    
    regrDF_train[shiftdays] = []
    regrDF_test [shiftdays] = []
    model_train [shiftdays] = []
    model_test  [shiftdays] = []
    
    for i in range(xval_count):
        regrDF_train[shiftdays].append(regrDF[shiftdays][samples != i])
        regrDF_test [shiftdays].append(regrDF[shiftdays][samples == i])
        
        model_train [shiftdays].append(smf.ols(formula = formula, data = regrDF_train[shiftdays][i]))
        model_test  [shiftdays].append(smf.ols(formula = formula, data = regrDF_test[shiftdays][i]))


generating DF and model for shiftdays = 5
generating DF and model for shiftdays = 6
generating DF and model for shiftdays = 7
generating DF and model for shiftdays = 8
generating DF and model for shiftdays = 9


In [9]:
for shiftdays in np.arange(5,10):
    
    print('shiftdays = {}'.format(shiftdays))
    
    for alpha in np.power(10,np.linspace(-6, -3.4, num = 14, endpoint = True)):
        print('   alpha = {:.6f}'.format(alpha))
        for i in range(xval_count):
            
            result_dict = CrossValidationFit( trainmodel = model_train[shiftdays][i],
                                              testmodel  = model_test[shiftdays][i],
                                              header     = {'shiftdays':shiftdays, 'alpha':alpha, 'iteration':i},
                                              alpha      = alpha)

            
            if allresults_rndXv_chunksize is None:
                allresults_rndXv_chunksize = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
            else:
                allresults_rndXv_chunksize = allresults_rndXv_chunksize.append(result_dict, ignore_index = True)

allresults_rndXv_chunksize.to_csv('results_Xval_rnd10_200411_5.csv')

shiftdays = 5
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
shiftdays = 6
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
shiftdays = 7
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
shiftdays = 8
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   a

In [10]:
# cross-validation with random sampling
# repetition 10 times

samples_dict = {}

for repetition in range(10):
    
    print('repetition = {}'.format(repetition))
    
    maxshift     = 10
    regrDF       = {}
    model_train  = {}
    model_test   = {}

    regrDF_train = {}
    regrDF_test  = {}

    allresults_rndXv_chunksize = None



    for shiftdays in range(5, 10):
        print('generating DF and model for shiftdays = {}'.format(shiftdays))
        regrDF[shiftdays], measurelist = GetRegressionDF( countrylist = measure_data.countrylist,
                                                          shiftdays   = shiftdays,
                                                          smooth      = True)

        countrylist = regrDF[shiftdays]['Country'].unique()
        formula     = 'Observable ~ C(Country) + ' + ' + '.join(measurelist)

        # generate random list of samples for 10x cross validation
        xval_count = 10
        datalen    = len(regrDF[shiftdays])
        chunklen   = np.ones(xval_count,dtype = np.int) * (datalen // xval_count)
        chunklen[:datalen%xval_count] += 1
        samples    = np.random.permutation(np.concatenate([i*np.ones(chunklen[i],dtype = np.int) for i in range(xval_count)]))
        
        samples_dict[repetition] = samples
        assert len(samples) == datalen

        regrDF_train[shiftdays] = []
        regrDF_test [shiftdays] = []
        model_train [shiftdays] = []
        model_test  [shiftdays] = []

        for i in range(xval_count):
            regrDF_train[shiftdays].append(regrDF[shiftdays][samples != i])
            regrDF_test [shiftdays].append(regrDF[shiftdays][samples == i])

            model_train [shiftdays].append(smf.ols(formula = formula, data = regrDF_train[shiftdays][i]))
            model_test  [shiftdays].append(smf.ols(formula = formula, data = regrDF_test[shiftdays][i]))

            
    for shiftdays in np.arange(5,10):

        print('model estimates, s = {}'.format(shiftdays))

        for alpha in np.power(10,np.linspace(-6, -3.4, num = 14, endpoint = True)):
            print('   alpha = {:.6f}'.format(alpha))
            for i in range(xval_count):

                result_dict = CrossValidationFit( trainmodel = model_train[shiftdays][i],
                                                  testmodel  = model_test[shiftdays][i],
                                                  header     = {'shiftdays':shiftdays, 'alpha':alpha, 'iteration':i},
                                                  alpha      = alpha)


                if allresults_rndXv_chunksize is None:
                    allresults_rndXv_chunksize = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
                else:
                    allresults_rndXv_chunksize = allresults_rndXv_chunksize.append(result_dict, ignore_index = True)

    allresults_rndXv_chunksize.to_csv('results_Xval_200412_{}.csv'.format(repetition))
pd.DataFrame({'rep{}'.format(s):np.array(samp) for s,samp in samples_dict.items()}).to_csv('samplevec_Xval_200412.csv')

repetition = 0
generating DF and model for shiftdays = 5


/usr/lib64/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/lib64/python3.7/site-packages/numpy/lib/function_base.py:1273: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


generating DF and model for shiftdays = 6
generating DF and model for shiftdays = 7
generating DF and model for shiftdays = 8
generating DF and model for shiftdays = 9
model estimates, s = 5
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 6
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 7
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   

   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 9
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
repetition = 5
generating DF and model for shiftdays = 5
generating DF and model for shiftdays = 6
generating DF and model for shiftdays = 7
generating DF and model for shiftdays = 8
generating DF and model for shiftdays = 9
model estimates, s = 5
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 6
   alp

   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 7
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 8
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 9
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398


In [ ]:
# cross-validation with random sampling
# repetition 10 times

samples_dict = {}

for repetition in range(10):
    
    print('repetition = {}'.format(repetition))
    
    maxshift     = 10
    regrDF40       = {}
    model_train40  = {}
    model_test40   = {}

    regrDF_train40 = {}
    regrDF_test40  = {}

    allresults_rndXv_chunksize40 = None



    for shiftdays in range(5, 10):
        print('generating DF and model for shiftdays = {}'.format(shiftdays))
        regrDF40[shiftdays], measurelist = GetRegressionDF( countrylist = measure_data.countrylist,
                                                          shiftdays   = shiftdays,
                                                          maxlen      = 40,
                                                          smooth      = True)

        countrylist = regrDF40[shiftdays]['Country'].unique()
        formula     = 'Observable ~ C(Country) + ' + ' + '.join(measurelist)

        # generate random list of samples for 10x cross validation
        xval_count = 10
        datalen    = len(regrDF40[shiftdays])
        chunklen   = np.ones(xval_count,dtype = np.int) * (datalen // xval_count)
        chunklen[:datalen%xval_count] += 1
        samples    = np.random.permutation(np.concatenate([i*np.ones(chunklen[i],dtype = np.int) for i in range(xval_count)]))
        
        samples_dict[repetition] = samples
        assert len(samples) == datalen

        regrDF_train40[shiftdays] = []
        regrDF_test40 [shiftdays] = []
        model_train40 [shiftdays] = []
        model_test40  [shiftdays] = []

        for i in range(xval_count):
            regrDF_train40[shiftdays].append(regrDF40[shiftdays][samples != i])
            regrDF_test40 [shiftdays].append(regrDF40[shiftdays][samples == i])

            model_train40 [shiftdays].append(smf.ols(formula = formula, data = regrDF_train40[shiftdays][i]))
            model_test40  [shiftdays].append(smf.ols(formula = formula, data = regrDF_test40[shiftdays][i]))

            
    for shiftdays in np.arange(5,10):

        print('model estimates, s = {}'.format(shiftdays))

        for alpha in np.power(10,np.linspace(-6, -3.4, num = 14, endpoint = True)):
            print('   alpha = {:.6f}'.format(alpha))
            for i in range(xval_count):

                result_dict = CrossValidationFit( trainmodel = model_train40[shiftdays][i],
                                                  testmodel  = model_test40[shiftdays][i],
                                                  header     = {'shiftdays':shiftdays, 'alpha':alpha, 'iteration':i},
                                                  alpha      = alpha)


                if allresults_rndXv_chunksize40 is None:
                    allresults_rndXv_chunksize40 = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
                else:
                    allresults_rndXv_chunksize40 = allresults_rndXv_chunksize40.append(result_dict, ignore_index = True)

    allresults_rndXv_chunksize40.to_csv('results_Xval_maxlen40_200412_{}.csv'.format(repetition))
pd.DataFrame({'rep{}'.format(s):np.array(samp) for s,samp in samples_dict.items()}).to_csv('samplevec_Xval_maxlen40_200412.csv')

repetition = 0
generating DF and model for shiftdays = 5
generating DF and model for shiftdays = 6
generating DF and model for shiftdays = 7
generating DF and model for shiftdays = 8
generating DF and model for shiftdays = 9
model estimates, s = 5
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 6
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alpha = 0.000025
   alpha = 0.000040
   alpha = 0.000063
   alpha = 0.000100
   alpha = 0.000158
   alpha = 0.000251
   alpha = 0.000398
model estimates, s = 7
   alpha = 0.000001
   alpha = 0.000002
   alpha = 0.000003
   alpha = 0.000004
   alpha = 0.000006
   alpha = 0.000010
   alpha = 0.000016
   alp

In [ ]:
allresults_rndXv.to_csv('results_Xval_rnd10_200411.csv')

In [ ]:
allresults_rndXv.head()